In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_27.csv')
pred2 = dd.read_csv('test_pred_28.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 35.6 s, sys: 3.28 s, total: 38.9 s
Wall time: 8.39 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 181 ms, sys: 275 ms, total: 456 ms
Wall time: 455 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 283 µs, sys: 36 µs, total: 319 µs
Wall time: 323 µs


In [7]:
coef_lgb = 0.7
coef_mlp = 0.3

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 288 ms, sys: 602 ms, total: 890 ms
Wall time: 887 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000228,0.298213,0.000388,0.206466,0.193364,0.000438,0.106910,0.000194,0.001862,0.018392,0.002771,0.169780,0.000294,0.000700,0.246693
3492886,130787971,0.000166,0.013110,0.000223,0.055730,0.215129,0.000264,0.019806,0.307609,0.000415,0.029630,0.000814,0.354066,0.000249,0.002787,0.238732
3492887,130787974,0.000535,0.020342,0.000128,0.826279,0.039241,0.000284,0.089637,0.000048,0.000144,0.001105,0.000403,0.018507,0.000177,0.003171,0.106709
3492888,130788053,0.000067,0.717149,0.000135,0.133362,0.006420,0.000137,0.009713,0.090632,0.000415,0.000328,0.000192,0.040198,0.000154,0.001100,0.164855
3492889,130788054,0.000202,0.104326,0.000298,0.360396,0.183922,0.000459,0.049215,0.000141,0.000683,0.016808,0.000854,0.281382,0.000600,0.000713,0.234415


In [13]:
%%time
test_ensemble.to_csv('test_pred_29.csv',index=False)

CPU times: user 1min 34s, sys: 1.47 s, total: 1min 36s
Wall time: 1min 36s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [1]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_29.csv -m "Message"

100%|███████████████████████████████████████| 1.06G/1.06G [40:22<00:00, 471kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification